# Tutorial DtaFrames y MLJ

Antes de ejecutar ninguna celda, por favor asegúrate de activar e instanciar el entorno de paquetes para el tutorial. Para ello, usa los ficheros `Project.toml` y y `Manifest.toml` que se distribuyen en la página de la asignatura junto a este notebook (graba todo en una carpeta, por ejemplo de nombre `TutorialML`). Tras tener todo preparado, puedes volver a este notebook y comenzar a ejecutar las celdas.

**Importante**: La siguiente celda prepara el entorno cargando los paquetes necesarios.

In [ ]:
using Pkg; Pkg.activate(""); Pkg.instantiate()


# 1. Carga de Datos

En esta sección mostramos dos formas de cargar datos fácilmente en Julia: 
1. Cargar un conjunto de datos estándar mediante `RDatasets.jl`.
1. Cargar un archivo local con `CSV.jl`.

## Usando RDatasets

La primera necesidad cuando se comienza a trabajar en ML son los datos, que deben verificar algunas condiciones para que se pueda extraer conocimiento acerca de las características de los mismos, de los modelos usados como hipótesis y del resto de técnicas que se aplican en esta área.

La tarea de recopilación de datos puede requerir mucha inversión de tiempo y recursos (humanos, por ejemplo). Por suerte, con el paso de los años se han creado multitud de herramientas para acceder a repositorios abiertos con muchos datasets disponibles para experimentar y aprender. Los dos más renombrables para nuestras necesidades (pero no los únicos) son: `RDatasets` y `OpenML`.

El hecho de que el dataset `Boston` forme parte de `MASS` se indica claramente en [esta lista](http://vincentarelbundock.github.io/Rdatasets/datasets.html). Aunque puede ser un poco lento, cargar un conjunto de datos mediante RDatasets es muy sencillo y cómodo, ya que no hay que preocuparse de establecer los nombres de las columnas, etc. La función `dataset` devuelve un objeto `DataFrame` del paquete [DataFrames.jl](https://github.com/JuliaData/DataFrames.jl), que veremos superficialmente en una sección posterior.

In [ ]:
using RDatasets
import DataFrames

boston = dataset("MASS", "Boston")

In [ ]:
typeof(boston)

## Usando CSV

El paquete [CSV.jl](https://github.com/JuliaData/CSV.jl) ofrece una forma eficaz de leer archivos CSV arbitrarios. 

En particular, la función `CSV.read` permite leer un archivo en este formato y devolver un DataFrame.

**Importante**: No confundir `CSV` y `DataFrames` como dos alternativas disjuntas: `CSV` es un formato de fichero, y `DataFrames` es un formato de memoria.

Supongamos que tenemos un fichero `foo.csv` en una ruta `fpath=joinpath("data", "foo.csv")` con el contenido:

```
col1,col2,col3,col4,col5,col6,col7,col8
,1,1.0,1,one,2019-01-01,2019-01-01T00:00:00,true
,2,2.0,2,two,2019-01-02,2019-01-02T00:00:00,false
,3,3.0,3.14,three,2019-01-03,2019-01-03T00:00:00,true
```

La siguiente celda crea un fichero con este contenido en un directorio temporal, para que podamos hacer el ejercicio de leerlo más adelante:

In [120]:
c = """
col1,col2,col3,col4,col5,col6,col7,col8
,1,1.0,1,one,2019-01-01,2019-01-01T00:00:00,true
,2,2.0,2,two,2019-01-02,2019-01-02T00:00:00,false
,3,3.0,3.14,three,2019-01-03,2019-01-03T00:00:00,true
"""

fpath, = mktemp()
write(fpath, c);

Ahora, podemos leero usando `CSV`:

In [ ]:
using CSV
data = CSV.read(fpath, DataFrames.DataFrame)

Ten en cuenta que podríamos pasar cualquier ruta válida del sistema, por ejemplo `CSV.read("path/to/file.csv")`. 

Los datos también se devuelven como un `Dataframe`:

In [ ]:
typeof(data)

Algunos de los argumentos útiles para `read` son:

* `header=` para especificar si hay una cabecera, o en qué línea está la cabecera. También permite especificar una cabecera concreta,
* `skipto=` para especificar cuántas filas saltarse antes de empezar a leer los datos,
* `limit=` para especificar el número máximo de filas a analizar,
* `missingstring=` para especificar una cadena o vector de cadenas que deben analizarse como valores perdidos,
* `delim=','` un carácter o cadena para especificar cómo se separan las columnas.

### Ejemplo 1

Supongamos que tenemos el siguiente conjunto de datos, cuyo contenido guardamos en un archivo en la ruta `fpath`.

In [123]:
c = """
3.26;0.829;1.676;0;1;1.453;3.770
2.189;0.58;0.863;0;0;1.348;3.115
2.125;0.638;0.831;0;0;1.348;3.531
3.027;0.331;1.472;1;0;1.807;3.510
2.094;0.827;0.86;0;0;1.886;5.390
3.222;0.331;2.177;0;0;0.706;1.819
3.179;0;1.063;0;0;2.942;3.947
3;0;0.938;1;0;2.851;3.513
2.62;0.499;0.99;0;0;2.942;4.402
2.834;0.134;0.95;0;0;1.591;3.021
2.405;0.134;0.843;0;0;1.769;3.210
2.728;0.223;0.953;0;0;1.591;2.371
2.512;0.223;0.929;1;0;1.769;3.919
2.834;0.134;1.237;0;0;1.859;3.030
2.819;0.331;1.271;0;1;0.981;2.736
2.126;0.251;1.114;0;0;0.143;2.157
2.834;0.134;1.322;0;0;1.199;2.413
3.014;0.56;1.781;0;0;-0.115;0.898
3.024;0.452;2.698;0;0;1.107;0.450
3.036;0.405;1.205;1;0;1.807;3.733
2.707;0.972;1.889;0;3;-1.169;2.976
2.978;1.246;1.103;0;1;3.988;6.535
3.111;0.732;0.923;0;0;4.068;5.643
"""
fpath, = mktemp()
write(fpath, c);

Como no tiene una cabecera, se la proporcionamos explícitamente:

In [ ]:
header = ["CIC0", "SM1_Dz", "GATS1i",
          "NdsCH", "NdssC", "MLOGP", "LC50"]
data = CSV.read(fpath, DataFrames.DataFrame, header=header)
first(data, 3)

### Ejemplo 2

Consideremos ahora este conjunto de datos, cuyo contenido guardamos en `fpath`.

In [125]:
c = """
1,0,1,0,0,0,0,1,0,1,1,?,1,0,0,0,0,1,0,0,0,0,1,67,137,15,0,1,1,1.53,95,13.7,106.6,4.9,99,3.4,2.1,34,41,183,150,7.1,0.7,1,3.5,0.5,?,?,?,1
0,?,0,0,0,0,1,1,?,?,1,0,0,1,0,0,0,1,0,0,0,0,1,62,0,?,0,1,1,?,?,?,?,?,?,?,?,?,?,?,?,?,?,1,1.8,?,?,?,?,1
1,0,1,1,0,1,0,1,0,1,0,0,0,1,1,0,0,0,0,1,0,1,1,78,50,50,2,1,2,0.96,5.8,8.9,79.8,8.4,472,3.3,0.4,58,68,202,109,7,2.1,5,13,0.1,28,6,16,1
1,1,1,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1,1,77,40,30,0,1,1,0.95,2440,13.4,97.1,9,279,3.7,0.4,16,64,94,174,8.1,1.11,2,15.7,0.2,?,?,?,0
1,1,1,1,0,1,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1,76,100,30,0,1,1,0.94,49,14.3,95.1,6.4,199,4.1,0.7,147,306,173,109,6.9,1.8,1,9,?,59,15,22,1
1,0,1,0,?,0,0,1,0,?,0,1,0,0,0,0,0,1,1,1,0,0,1,75,?,?,1,1,2,1.58,110,13.4,91.5,5.4,85,3.4,3.5,91,122,242,396,5.6,0.9,1,10,1.4,53,22,111,0
1,0,0,0,?,1,1,1,0,0,1,0,?,0,0,0,0,0,0,0,0,0,1,49,0,0,0,1,1,1.4,138.9,10.4,102,3.2,42000,2.35,2.72,119,183,143,211,7.3,0.8,5,2.6,2.19,171,126,1452,0
1,1,1,0,?,0,0,1,0,1,1,?,0,0,0,0,0,0,1,1,1,0,1,61,?,20,3,1,1,1.46,9860,10.8,92,3,58,3.1,3.2,79,108,184,300,7.1,0.52,2,9,1.3,42,25,706,0
1,1,1,0,0,0,0,1,0,1,1,0,0,1,0,0,0,?,1,1,0,0,1,50,100,32,1,1,2,3.14,8.8,11.9,107.5,4.9,70,1.9,3.3,26,59,115,63,6.1,0.59,1,6.4,1.2,85,73,982,1
1,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,43,100,0,0,1,1,1.12,1.8,11.8,87.8,5100,193000,4.2,0.5,71,45,256,303,7.1,0.59,1,9.3,0.7,?,?,?,1
1,0,1,0,0,0,1,1,?,?,0,0,0,0,0,0,0,?,1,1,0,0,1,41,?,?,0,1,2,1.05,100809,13,94.2,5.7,196,4.4,3,90,334,494,236,7.6,0.8,5,?,1.1,?,?,?,0
1,0,1,0,0,0,1,1,1,0,0,0,0,1,0,0,0,?,0,1,0,0,1,74,?,0,0,1,1,1.33,86,15.7,96.7,4,61,3.7,1.3,132,168,113,154,?,7.6,5,1.9,0.3,144,41,277,1
1,0,1,0,0,0,0,1,0,1,1,0,0,1,0,0,?,?,1,1,1,0,0,66,?,30,0,1,1,1.53,60,13.3,90.1,5.5,207000,4.4,8.5,25,36,35,74,8.5,0.73,1,5,0.8,?,?,?,1
1,?,0,0,0,0,1,1,?,?,0,0,0,0,0,0,0,0,0,0,0,0,1,56,0,?,0,1,1,1.2,6.6,13.7,93.8,4.1,91000,4.5,1,103,96,205,70,8.8,0.88,1,22,?,82,24,?,1
1,0,1,0,0,0,0,1,0,?,1,0,0,1,0,0,?,1,1,1,0,0,1,63,?,?,2,2,2,1.25,29,13.5,93,6,128,3.15,10.5,76,116,165,163,7.3,1.07,4,4.5,4.5,197,84,302,1
0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,1,0,1,41,100,0,1,1,2,1.61,4.6,10.2,89.6,5.5,161,3.1,3.1,24,57,163,176,5,0.8,2,2.6,1.3,25,13,60,1
1,0,1,0,0,0,0,1,?,1,1,?,1,0,0,0,?,?,1,1,1,0,1,72,?,?,3,2,1,2.14,60,12.1,99.2,5,58,2.4,9.8,69,63,201,235,6.2,0.96,2,2,2.9,136,95,767,0
1,1,1,0,0,0,0,1,0,1,0,0,?,0,0,0,?,1,1,1,1,1,1,60,100,60,2,1,1,1.05,9.2,10.3,103.7,5.4,159,3.8,0.5,56,91,459,146,5.4,1.23,5,13.5,3.8,187,58,443,1
1,?,1,0,0,0,0,1,?,1,0,?,0,1,1,0,0,1,0,0,0,0,1,64,200,78,1,1,1,1.13,8.8,14.9,94.8,6.3,137,4.3,0.9,16,23,82,180,6.5,4.95,1,5.4,0.9,144,49,295,1
1,1,1,0,0,0,0,1,?,?,0,0,0,1,0,0,0,1,1,1,1,0,1,75,500,?,0,1,3,1.44,34,15.9,103.4,9600,101000,3.4,3.4,27,87,260,147,6.3,0.9,5,2.3,1.6,67,34,774,0
"""
fpath, = mktemp()
write(fpath, c);

No tiene cabecera, y además tiene valores perdidos indicados por `?`.

In [ ]:
data = CSV.read(fpath, DataFrames.DataFrame, header=false, missingstring="?")
first(data[:, 1:5], 3)

# 2. Dataframes

Este tutorial no pretende ser una introducción completa, sino solo enfatizar algunas funcionalidades que son particularmente útiles en un contexto clásico de aprendizaje automático.

Podemos crear un dataframe directamente, indicando por columnas qué valores tiene:

In [ ]:
using DataFrames
DataFrame(a=1:4, b=["M", "F", "F", "M"])

... hay muchas formas de construir un dataframe:

In [ ]:
DataFrame([(a=1, b=0), (a=2, b=0)])     # constructor a partir de una tabla
DataFrame("a" => 1:2, "b" => 0)         # ... de un par
DataFrame([:a => 1:2, :b => 0])         # ... de un vector de pares
DataFrame(Dict(:a => 1:2, :b => 0))     # ... de un diccionario
DataFrame([[1, 2], [0, 0]], [:a, :b])   # ... de un vector de vectores
DataFrame([1 0; 2 0], :auto)            # ... de una matriz

Vamos a utilizar el conjunto de datos `Boston` que ya usamos en la sección anterior, que es suficientemente sencillo y completo (repetimos líneas de código para marcar su importancia, pero no porque sean necesarias):

In [ ]:
using RDatasets

boston = dataset("MASS", "Boston")
typeof(boston)

Intuitivamente, un DataFrame no es más que una envoltura alrededor de una serie de columnas, cada una de las cuales es en realidad un `Vector` de algún tipo con un nombre. Podemos acceder a los nombres de las columnas con la función `names`:

In [ ]:
names(boston)
# propertynames(boston) # para obtener los nombres como símbolos

Y a las primeras filas del dataframe utilizando `first` y especificando un número de filas:

In [ ]:
first(boston, 4)

Para acceder a cualquiera de esas columnas basta usar `.colname`, esto devuelve un vector al que puedes acceder como a cualquier vector de Julia:

In [ ]:
boston.Crim[1:5] # es lo mismo que: boston[1:5, :Crim]

O incluso al dataframe como si fuera una gran matriz:

In [ ]:
boston[3, 5]

o especificando un rango de filas/columnas:

In [ ]:
boston[1:5, [:Crim, :Zn]]

o, similarmente,

In [ ]:
boston[1:5, 1:2]

La función `select` es muy práctica para obtener sub-dataframes, indicando las columnas de interés:

In [ ]:
b1 = select(boston, [:Crim, :Zn, :Indus])
first(b1, 2)

o usando la sintaxis `Not` para indicar las que no queremos incluir:

In [ ]:
b2 = select(boston, Not(:NOx))
first(b2, 2)

Por último, recuerda que, como es habitual en Julia, si deseas eliminar columnas puedes utilizar `select!`, que mutará el dataframe en su lugar:

In [ ]:
select!(b1, Not(:Crim))
first(b1, 2)

### Describir los datos del dataframe

`StatsBase` ofrece una práctica función `describe` que puedes utilizar en un DataFrame para obtener una visión general de los datos:

In [ ]:
using StatsBase
describe(boston, :min, :max, :mean, :median, :std)

Puedes pasar una serie de símbolos a la función `describe` para indicar qué estadísticas calcular para cada característica/columna:

* `mean`, `std`, `min`, `max`, `median`, `first`, `last` se explican por sí solas.
* `q25`, `q75` corresponden al percentil 25 y 75, respectivamente.
* También se pueden utilizar `eltype`, `nunique`, `nmissing`.

También puedes pasar una función personalizada con un par `name = > function`. Por ejemplo:

In [ ]:
mi_media(x) = sum(abs.(x)) / length(x)
d = describe(boston, :mean, :median, mi_media => :mi_media)
first(d, 3)

La función `describe` devuelve un objeto derivado con una fila por cada característica de los datos, y una columna por estadística requerida. 

Además de `StatsBase`, el paquete `Statistics` ofrece una serie de funciones adicionales muy útiles para el análisis de datos.

In [139]:
using Statistics

### Convirtiendo los datos

Para manipular el contenido del dataframe como una gran matriz, podemos utilizar el paquete interno de Juia `convert`. Por ejemplo:

In [ ]:
mat = Matrix(boston)
mat[1:3, 1:3]

### Añadir columnas

Añadir una columna a un dataframe es muy fácil. Por ejemplo, para crear una columna que hace uso de otras columnas del dataframe:

In [141]:
boston.Crim_x_Zn = boston.Crim .* boston.Zn;

Recuerda que se pueden eliminar columnas o hacer subselecciones con `select` y `select!`.

### Valores Faltantes

Vamos a cargar un dataset con valores faltantes:

In [ ]:
mao = dataset("gap", "mao")
describe(mao, :nmissing)

Faltan muchos valores... algo que podría impedir el uso de algunas funciones sobre esas columnas, ya que quizás no puedan ejecutarse si hay filas con valor `missing`. Por ejemplo:

In [ ]:
std(mao.Age)

La función `skipmissing` puede ayudar a contrarrestar esto fácilmente:

In [ ]:
std(skipmissing(mao.Age))

## Split-Apply-Combine

Volvamos a cambiar de dataset, uno que veremos más de una vez en el curso y que tiene un contenido sencillo con el que se facilita la comprensión de las siguientes funcionalidades:

In [ ]:
iris = dataset("datasets", "iris")
first(iris, 3)

### `groupby`

La función `groupby` permite formar sub-dataframes correspondientes a grupos de filas, algo muy práctico para ejecutar análisis específicos para grupos concretos sin tener que copiar los datos.

El uso básico es `groupby(df, cols)` donde `cols` especifica una o varias columnas a utilizar para la agrupación.

Consideremos un ejemplo sencillo sobre el dataset `iris`, que tiene una columna `Species` con 3 valores:

In [ ]:
unique(iris.Species)

Podemos formar vistas para cada una de ellas:

In [147]:
gdf = groupby(iris, :Species);

El objeto `gdf` corresponde ahora a **vistas** del dataframe original para cada una de las 3 especies; la primera especie es `«setosa»` con:

In [ ]:
subdf_setosa = gdf[1]
describe(subdf_setosa, :min, :mean, :max)

Ten en cuenta que `subdf_setosa` es un `SubDataFrame`, lo que significa que es sólo una vista del dataframe padre `iris`; si modificas ese dataframe padre, el sub dataframe también se modifica.

### `combine`

La función `combine` permite derivar un nuevo dataframe a partir de transformaciones de uno ya existente.

Por ejemplo:

In [ ]:
df = DataFrame(a=1:3, b=4:6)
combine(df, :a => sum, nrow)

lo que ha ocurrido aquí es que el DataFrame derivado tiene dos columnas obtenidas respectivamente por (1) calcular la suma de la primera columna y (2) aplicar la función `nrow` sobre el `df`.

La transformación puede producir uno o varios valores, `combine` intentará concatenar estas columnas como pueda, por ejemplo:

In [ ]:
foo(v) = v[1:2]
combine(df, :a => maximum, :b => foo)

donde el valor máximo de `a` se ha copiado dos veces para que las dos columnas tengan el mismo número de filas.

In [ ]:
bar(v) = v[end-1:end]
combine(df, :a => foo, :b => bar)

### `combine` con `groupby`

Combinar `groupby` con `combine` es muy útil. Por ejemplo, es posible que desees calcular estadísticas entre grupos para diferentes variables:

In [ ]:
combine(groupby(iris, :Species), :PetalLength => mean)

descompongamos esto:

1. el `grroupby(iris, :Species)` crea grupos usando la columna `:Species` (que tiene los valores `setosa`, `versicolor`, `virginica`),
2. `combine` crea un dataframe derivado aplicando la función `mean` a la columna `:PetalLength`,
3. como hay tres grupos, obtenemos una columna (media de `LongitudPétalos`) y tres filas (una por grupo).


Puedes hacer esto para varias columnas/estadísticas a la vez y dar nuevos nombres de columna a los resultados:

In [ ]:
gdf = groupby(iris, :Species)
combine(gdf, :PetalLength => mean => :MPL, :PetalLength => std => :SPL)

aquí asignamos los nombres `:MPL` y `:SPL` a las columnas derivadas.
Si quieres aplicar algo en todas las columnas aparte de la de agrupación, usar `names` y `Not` es muy útil:

In [ ]:
combine(gdf, names(iris, Not(:Species)) .=> std)

donde

In [ ]:
names(iris, Not(:Species))

y observa el uso de `.` en `.=>` para indicar que aplicamos la función sobre cada columna.

# 3. Datos Categóricos

## Definiendo un vector categórico

In [ ]:
using CategoricalArrays

v = categorical(["AA", "BB", "CC", "AA", "BB", "CC"])

Declara un vector categórico, es decir, un vector cuyas entradas se espera que representen un grupo o categoría. Puedes recuperar las etiquetas de grupo utilizando `levels`:

In [ ]:
levels(v)

que, por defecto, devuelve las etiquetas en orden lexicográfico.

## Trabajando con categóricas

### Categorías ordenadas

Puedes especificar que las categorías estén *ordenadas* especificando `ordered=true`, el orden entonces sigue el de los niveles. Si quieres cambiar ese orden, tienes que usar la función `niveles!`.

Veamos dos ejemplos:

In [ ]:
v = categorical([1, 2, 3, 1, 2, 3, 1, 2, 3], ordered=true)

levels(v)

Aquí el orden lexicográfico coincide con lo que queremos así que no hace falta cambiarlo, como hemos especificado que las categorías están ordenadas podemos hacerlo:

In [ ]:
v[1] < v[2]

Veamos ahora otro ejemplo

In [ ]:
v = categorical(["high", "med", "low", "high", "med", "low"], ordered=true)

levels(v)

Los niveles siguen el orden lexicográfico, que no es el que deseamos:

In [ ]:
v[1] < v[2]

Para volver a especificar el orden tenemos que utilizar `niveles!`:

In [ ]:
levels!(v, ["low", "med", "high"])

ahora las cosas están bien ordenadas:

In [ ]:
v[1] < v[2]

### Valores Faltantes

También puedes tener un vector categórico con valores faltantes:

In [164]:
v = categorical(["AA", "BB", missing, "AA", "BB", "CC"]);

que no cambia los niveles:

In [ ]:
levels(v)

# 4. Tipos Científicos

## Tipos de Máquina vs Tipos Científicos

### ¿Porqué es necesaria una distinción?

Al analizar datos, es importante distinguir entre

* _cómo se codifican los datos_ (por ejemplo, `Int`), y
* _cómo deben interpretarse los datos_ (por ejemplo, una etiqueta de clase, un recuento, ...)

La forma en que se codifican los datos se denomina **tipo de máquina**, mientras que la forma en que se interpretan los datos se denomina **tipo científico**.

En algunos casos, esto puede ser inequívoco, por ejemplo, un vector de valores de coma flotante normalmente se interpretará como una característica continua (por ejemplo: pesos, velocidades, temperaturas, ...).

En muchos otros casos, sin embargo, puede haber ambigüedades, enumeramos algunos ejemplos a continuación:

* Un vector de `Int`, por ejemplo `[1, 2, ...]`, que debe interpretarse como etiquetas categóricas,
* Un vector de `Int`, por ejemplo `[1, 2, ...]`, que debe interpretarse como datos de recuento,
* Un vector de `String` p. ej. `["Alto", "Bajo", "Alto", ...]` que debe interpretarse como etiquetas categóricas ordenadas,
* Un vector de `String` por ejemplo `["Juan", "María", ...]` que no deben interpretarse como datos informativos,
* Un vector de puntos flotantes `[1.5, 1.5, -2.3, -2.3]` que deben interpretarse como datos categóricos (por ejemplo, los pocos valores posibles de algún parámetro), etc.

### Los Tipos Científicos

El paquete [ScientificTypes.jl](https://github.com/JuliaAI/ScientificTypes.jl) define una jerarquía de tipos básica que puede utilizarse para indicar cómo debe interpretarse una característica determinada; en particular:

```plaintext
Found
├─ Known
│  ├─ Textual
│  ├─ Finite
│  │  ├─ Multiclass
│  │  └─ OrderedFactor
│  └─ Infinite
│     ├─ Continuous
│     └─ Count
└─ Unknown
```

Una *convención de tipos científicos* es una implementación específica que indica cómo pueden relacionarse los tipos máquina con los tipos científicos. También puede proporcionar funciones de ayuda para convertir datos a un determinado tipo científico.

La convención utilizada en MLJ está implementada en [ScientificTypes.jl](https://github.com/JuliaAI/ScientificTypes.jl).
Esto es lo que utilizaremos a lo largo de este documento; nunca necesitarás utilizar ScientificTypes.jl  directamente, a menos que quieras implementar tu propia convención de tipos científicos.


### Inspeccionando el Tipo Científico

La función `schema`

In [ ]:
using RDatasets
using ScientificTypes

boston = dataset("MASS", "Boston")
sch = schema(boston)

En estos casos, la mayoría de las variables tienen un tipo (máquina) `Float64` y su interpretación por defecto es `Continuous`.
También hay `:Chas`, `:Rad` y `:Tax` que tienen un tipo (máquina) `Int64` y su interpretación por defecto es `Count`.

Mientras que la interpretación como `Continuo` suele estar bien, la interpretación como `Count` necesita un poco más de atención.
Por ejemplo, ten en cuenta que:

In [ ]:
unique(boston.Chas)

por lo que, aunque tiene un tipo máquina `Int64` y, en consecuencia, una interpretación por defecto de `Count`, sería más apropiado interpretarlo como un `OrderedFactor`.

### Cambiando el Tipo Científico

Para volver a especificar el tipo o tipos de características de un conjunto de datos, puedes utilizar la función `coerce` y especificar pares de nombre de variable y tipo científico:

In [168]:
boston2 = coerce(boston, :Chas => OrderedFactor);

el efecto es convertir la columna `:Chas` en un vector categórico ordenado:

In [ ]:
eltype(boston2.Chas)

correspondiente al tipo de código `OrderedFactor`:

In [ ]:
elscitype(boston2.Chas)

También puedes especificar varios pares de una sola vez con `coerce`:

In [171]:
boston3 = coerce(boston, :Chas => OrderedFactor, :Rad => OrderedFactor);

### String y Unknown

Si una característica de tu dataset tiene elementos String, entonces el scitype por defecto es `Textual`; puedes elegir eliminar tales columnas o coaccionarlas para que sean categóricas:

In [ ]:
feature = ["AA", "BB", "AA", "AA", "BB"]
elscitype(feature)

que puedes coaccionar:

In [ ]:
feature2 = coerce(feature, Multiclass)
elscitype(feature2)

## Ayudas y Trucos

### Coarción Tipo a Tipo

En algunos casos, querrás reinterpretar todas las características interpretadas actualmente como un tipo `S1` en otro tipo `S2`.
Por ejemplo, si algunas características se interpretan actualmente como `Count` porque su tipo original era `Int`, pero quieres considerarlas todas como `Continuous`:

In [ ]:
data = select(boston, [:Rad, :Tax])
schema(data)

vamos a coaccionar desde `Count` a `Continuous`:

In [ ]:
data2 = coerce(data, Count => Continuous)
schema(data2)

### Autotype

Una última herramienta útil es `autotype`, que permite especificar *reglas* para definir automáticamente la interpretación de las características.
Puedes codificar tus propias reglas pero hay tres útiles que están precodificadas:

* la regla `:few_to_finite` que comprueba cuántas entradas únicas hay en un vector y si hay "pocas" (few) sugiere un tipo categórico,
* la regla `:discrete_to_continuous` convierte `Integer` o `Count` en `Continuous`.
* la regla `:string_to_multiclass` que devuelve `Multiclass` para cualquier columna de tipo cadena.

Por ejemplo:

In [ ]:
boston3 = coerce(boston, autotype(boston, :few_to_finite))
schema(boston3)

# 5. Procesamiento

## Más sobre procesamiento de datos

Este tutorial utiliza el conjunto de datos Global Power Plants Dataset del World Resources Institute para explorar el preprocesamiento de datos en Julia. El conjunto de datos se crea a partir de múltiples fuentes y se actualiza continuamente, lo que significa que hay muchos datos que faltan, caracteres no estándar, etc.

In [177]:
import MLJ: schema, std, mean, median, coerce, coerce!, scitype
using DataFrames
using UrlDownload

Importar datos:

In [178]:
raw_data = urldownload("https://github.com/tlienart/DataScienceTutorialsData.jl/blob/master/data/wri_global_power_plant_db_be_022020.csv?raw=true")
data = DataFrame(raw_data);

Este dataset contiene información sobre centrales eléctricas de varios países de todo el mundo.

El nivel de desagregación es la central eléctrica. Para cada central, hay información sobre su nombre, localización, capacidad y muchas otras características.

La función schema nos permite obtener una visión general rápida de las variables que contiene, incluidos sus tipos de máquina y científicos.

In [ ]:
schema(data)

Vemos que un pequeño número de características tienen valores para todas las plantas (es decir, para cada fila) presentes en el conjunto de datos.

Sin embargo, (i) varias características tienen valores perdidos (Union{Missing, _.type}) y (ii) no nos interesa trabajar con todas estas características.
En concreto, no nos interesa el origen de la información presente en el dataset ni los datos de generación.

Por lo tanto, eliminamos todas las columnas que contienen el origen de la información.

Definimos una función `is_active()` que devolverá un valor booleano `TRUE` si el nombre de la columna NO contiene (`!`) ninguna de las cadenas «fuente» o «generación».

Nótese la conversión de los nombres de columna de `:Symbol` a `:string` ya que la función `occursing` sólo acepta cadenas como argumentos.

In [180]:
is_active(col) = !occursin(r"source|generation", string(col))
active_cols = [col for col in names(data) if is_active(col)]
select!(data, active_cols);

También eliminamos otras columnas no deseadas y echamos un vistazo a nuestro "nuevo" dataframe.

In [ ]:
select!(data, Not([:wepp_id, :url, :owner]))
schema(data)

El resto de variables tienen dos tipos científicos diferentes: Continuas, Textuales.

De las que podemos obtener una visión general.

In [ ]:
describe(data)

# La función describe() muestra que hay varias características con valores faltantes.

*Nota: la función `describe()` es del paquete `DataFrames` (y no funcionará con otras tablas que no sean DataFrames), mientras que `schema()` es del paquete MLJ.

Vamos a jugar con los datos de capacidad, para los que no hay valores faltantes. Creamos un sub-dataframe y agregamos sobre ciertas dimensiones ( country y primary_fuel)

In [ ]:
capacity = select(data, [:country, :primary_fuel, :capacity_mw]);
first(capacity, 5)

Este dataframe contiene varios subgrupos (country y technology type) y sería interesante obtener agregados de datos por subgrupo.

Para obtener una `vista` del DataFrame por subgrupo, podemos utilizar la función `groupby`.

In [184]:
cap_gr = groupby(capacity, [:country, :primary_fuel]);

Si queremos agregar a nivel de country-fuel-type y calcular estadísticas de resumen a este nivel, podemos utilizar la función `combine` en el GroupedDataFrame que acabamos de crear.

Esta función toma como argumentos el GroupedDataFrame, el símbolo de la columna sobre la que aplicar la medida elegida.

In [ ]:
cap_mean = combine(cap_gr, :capacity_mw => mean)
cap_sum = combine(cap_gr, :capacity_mw => sum)
first(cap_sum, 3)

Ahora vamos a representar algunos de estos datos agregados para una selección de países, por país y tipo de tecnología:

In [ ]:
ctry_selec = r"BEL|FRA|DEU"
tech_selec = r"Solar"

cap_sum_plot = cap_sum[occursin.(ctry_selec, cap_sum.country) .& occursin.(tech_selec, cap_sum.primary_fuel), :]

Antes de trazar, también podemos ordenar los valores por orden decreciente utilizando `sort!()`.

In [ ]:
sort!(cap_sum_plot, :capacity_mw_sum, rev=true)

using Plots

Plots.bar(cap_sum_plot.country, cap_sum_plot.capacity_mw_sum, legend=false)

Ahora que tenemos la capacidad total por país y tipo de tecnología, utilicémosla para calcular la cuota de cada tecnología en la capacidad total. Para ello, crearemos primero un dataframe que contenga la capacidad total a nivel de país, siguiendo los mismos pasos anteriores.

In [188]:
cap_sum_ctry_gd = groupby(capacity, [:country]);
cap_sum_ctry = combine(cap_sum_ctry_gd, :capacity_mw => sum);

A continuación, unimos este dataframe con el desagregado, lo que requiere que convirtamos los dos GroupedDataFrame en DataFrames.

In [189]:
cap_sum = DataFrame(cap_sum);
cap_sum_ctry = DataFrame(cap_sum_ctry);
cap_share = leftjoin(cap_sum, cap_sum_ctry, on = :country, makeunique = true)
cap_share.capacity_mw_share = cap_share.capacity_mw_sum ./ cap_share.capacity_mw_sum_1;

Volvamos a visualizar nuestro dataframe, que ahora incluye la columna `capacity_mw_share`.

Analicemos ahora las características que presentan algunos valores faltantes. Supongamos que queremos calcular la edad de cada planta (redondeada a años completos). 

Nos enfrentamos a dos problemas: En primer lugar, no todas las instalaciones indican el año de puesta en servicio. Tenemos que evaluar la representatividad de las instalaciones de las que se dispone de datos con respecto al conjunto de datos completo. Una forma de contar los valores que faltan es:

In [ ]:
nMissings = length(findall(x -> ismissing(x), data.commissioning_year))

Esto representa aproximadamente la mitad de nuestras observaciones

In [ ]:
nMissings_share = nMissings/size(data)[1]

En segundo lugar, el año de puesta en servicio no se indica como un número entero. También se indican fracciones de años. 

Como resultado, el tipo de máquina de `data.commissioning_year` es Float64.

In [ ]:
typeof(data.commissioning_year)

Antes de calcular la edad media, eliminemos los valores que faltan.

In [193]:
data_nmiss = dropmissing(data, :commissioning_year);

Y redondea el año al número entero más próximo. Podemos hacerlo utilizando la función `round` y una función de asignación en la columna relevante del DataFrame.

In [194]:
map!(x -> round(x, digits=0), data_nmiss.commissioning_year, data_nmiss.commissioning_year);

# Ahora podemos calcular la edad de cada planta (conviene recordar que el dataset sólo contiene plantas activas)

current_year = fill!(Array{Float64}(undef, size(data_nmiss)[1]), 2020);
data_nmiss[:, :plant_age] = current_year - data_nmiss[:, :commissioning_year];

Como falta el año de puesta en servicio para aproximadamente la mitad de las plantas del dataset (17340, véase la descripción de los datos más arriba) y que los valores que faltan se propagan, la edad de la planta sólo estará disponible para 33643-17340 plantas. 

Veamos cuáles son las edades media y mediana de las plantas de las que tenemos datos:

In [ ]:
mean_age = mean(skipmissing(data_nmiss.plant_age))
median_age = median(skipmissing(data_nmiss.plant_age))

Y representamos esta información a un gráfico de frecuencias de las observaciones de la edad de la planta:

In [ ]:
histogram(data_nmiss.plant_age, color="blue",  bins=100, label="Frecuencia Edad de la Planta",
          normalize=:pdf, alpha=0.5, xlim=(0,130))
vline!([mean_age], linewidth=2, color="red", label="Edad Media")
vline!([median_age], linewidth=2, color="orange", label="Mediana de Edad")

Asegúrate de que todas las columnas pasadas, excepto las dimensiones de agregación, son del tipo `Float` o `Int`, de lo contrario la ejecución de la función fallará.

In [ ]:
age = select(data_nmiss, [:country, :primary_fuel, :plant_age])
age_mean = combine(groupby(age, [:country, :primary_fuel]), :plant_age => mean)

coal_means = age_mean[occursin.(ctry_selec, age_mean.country) .& occursin.(r"Coal", age_mean.primary_fuel), :]
gas_means = age_mean[occursin.(ctry_selec, age_mean.country) .& occursin.(r"Gas", age_mean.primary_fuel), :]

In [ ]:
p1 = Plots.bar(coal_means.country, coal_means.plant_age_mean, ylabel="Edad", title="Carbón")
p2 = Plots.bar(gas_means.country, gas_means.plant_age_mean, title="Gas")

plot(p1, p2, layout=(1, 2), size=(900,600), plot_title="Edad media de las Plantas por país y Tecnología")

# 6. Selección del Modelo

## Datos y su Interpretación

### Tipo Máquina y Tipo Científico

In [ ]:
using RDatasets
using MLJ
iris = dataset("datasets", "iris")

first(iris, 3) |> pretty

Observa que debajo de cada nombre de columna hay dos _tipos_ dados: el primero es el _tipo máquina_ y el segundo es el _tipo científico_.

* Tipo máquina**: es el tipo Julia en el que están codificados los datos, por ejemplo `Float64`,
* **tipo científico**: es un tipo correspondiente a cómo los datos deben ser _interpretados_, por ejemplo `Multiclase{3}`.

Si quieres especificar un tipo científico distinto del inferido, puedes hacerlo utilizando la función `coerce` junto con pares de nombres de columna y tipos científicos:

In [ ]:
iris2 = coerce(iris, :PetalWidth => OrderedFactor)
first(iris2[:, [:PetalLength, :PetalWidth]], 1) |> pretty

### Desempaquetamiento de Datos

La función `unpack` ayuda a especificar el objetivo y la entrada para una tarea de regresión o clasificación

In [ ]:
y, X = unpack(iris, ==(:Species), colname -> true)
first(X, 1) |> pretty

Los dos argumentos tras los dataframes deben entenderse como _funciones_ sobre nombres de columnas que especifican los datos de destino y de entrada respectivamente.

Veamos con más detalle lo que utilizamos aquí:

* `==(:Species)` es una forma abreviada de especificar que el objetivo debe ser la columna con nombre igual a `:Species`,
* `colname -> true` indica que cualquier otra columna debe ser tomada como entrada (sería lo mismo que no ponerlo, en ese caso se consideran todas las restantes).

Probemos con otra:

In [ ]:
y, X = unpack(iris, ==(:Species), !=(:PetalLength))
first(X, 1) |> pretty

También puedes utilizar la abreviatura `@load_iris` para estos ejemplos comunes:

In [203]:
X, y = @load_iris;

## Selección del Modelo

### Búsqueda del Modelo

En MLJ, un _modelo_ es una estructura que almacena los _hiperparámetros_ del algoritmo de aprendizaje indicado por el nombre de la estructura (y sólo eso).

Hay varios modelos disponibles en MLJ, normalmente gracias a paquetes externos que interactúan con MLJ.

Para ver cuáles son apropiados para los datos que tienes y su interpretación científica, puedes utilizar la función `models` junto con la función `matching`; veamos específicamente los modelos que admiten una salida probabilística:

In [ ]:
for m in models(matching(X, y))
    if m.prediction_type == :probabilistic
        println(rpad(m.name, 30), "($(m.package_name))")
    end
end

### Carga de un Modelo

La mayoría de los modelos se implementan fuera del ecosistema MLJ; por lo tanto, tienes que _cargar los modelos_ usando el comando `@load`.

**Nota**: _debes_ tener disponible en tu entorno el paquete desde el que se carga el modelo (en este caso `[DecisionTree.jl]`) de lo contrario MLJ no podrá cargar el modelo.

Por ejemplo, supongamos que quieres cargar un clasificador K-Nearest Neighbours:

In [ ]:
knc = @load KNeighborsClassifier

En algunos casos, puede haber varios paquetes que ofrezcan el mismo modelo, por ejemplo `LinearRegressor` es ofrecido tanto por `[GLM.jl]` como por `[ScikitLearn.jl]` por lo que tendrás que especificar el paquete que deseas utilizar añadiendo `pkg="ThePackage"` en el comando load:

In [ ]:
linreg = @load LinearRegressor pkg=GLM

# 7. Entrenamiento y Predicción

## Pasos Preliminares

### Datos

Como ya hemos hecho antes, vamos a cargar el modelo Iris:

In [207]:
using MLJ
import Statistics
using PrettyPrinting
using StableRNGs

X, y = @load_iris;

y vamos a cargar también `DecisionTreeClassifier`:

In [ ]:
DecisionTreeClassifier = @load DecisionTreeClassifier pkg=DecisionTree
tree_model = DecisionTreeClassifier()

### Máquina MLJ 

En MLJ, un *modelo* es un objeto que sólo sirve como contenedor de los hiperparámetros del modelo.

Una *máquina* es un objeto que envuelve tanto un modelo como datos y puede contener información sobre el modelo *entrenado*; pero no *ajusta* el modelo por sí misma.

Sin embargo, sí comprueba que el modelo es compatible con el tipo científico de los datos y avisará en caso contrario.

In [ ]:
tree = machine(tree_model, X, y)

Se utiliza una máquina tanto para el modelo supervisado como para el no supervisado. 

Comenzaremos con un ejemplo para el modelo supervisado y luego seguimos con el caso no supervisado.

## Entrenamiento y Testeo de un Modelo Supervisado

Ahora que has declarado el modelo que quieres considerar y los datos, queda el paso estándar de entrenamiento y test para un algoritmo de aprendizaje supervisado.

### División de los Datos

Para dividir los datos en un conjunto de *entrenamiento* y *test*, puedes utilizar la función `partition` para obtener los índices de los datos que deben considerarse como datos de entrenamiento o de test:

In [ ]:
rng = StableRNG(566)
train, test = partition(eachindex(y), 0.7, shuffle=true, rng=rng)
test[1:3]

### Ajuste y testeo de la máquina

Para ajustar la máquina, puedes utilizar la función `fit!` especificando las filas que se utilizarán para el entrenamiento:

In [ ]:
fit!(tree, rows=train)

Ten en cuenta que esto **modifica** la máquina, que ahora contiene los parámetros entrenados del árbol de decisión. 

Puedes inspeccionar el resultado del ajuste con el método `fitted_params`:

In [ ]:
fitted_params(tree) |> pprint

Este `fitresult` variará de un modelo a otro, aunque los clasificadores suelen dar una tupla en la que el primer elemento corresponde al ajuste y el segundo lleva la cuenta de cómo se nombran las clases (para que las predicciones puedan nombrarse adecuadamente).

Ahora puedes usar la máquina para hacer predicciones con la función `predict` especificando las filas que se usarán para la predicción:

In [ ]:
ŷ = MLJ.predict(tree, rows=test)
@show ŷ[1]

Ten en cuenta que la salida es *probabilística*, efectivamente un vector con una puntuación para cada clase.

Podrías obtener el modo usando la función `mode` en `ŷ` o usando `predict_mode`:

In [ ]:
ȳ = predict_mode(tree, rows=test)
@show ȳ[1]
@show mode(ŷ[1])

Para medir la discrepancia entre `ŷ` y `y` se puede utilizar la entropía cruzada:

In [ ]:
mce = cross_entropy(ŷ, y[test])
round(mce, digits=4)

## Modelos No Supervisados

Los modelos no supervisados definen un método `transform`, y opcionalmente pueden implementar un método `inverse_transform`. 

Como en el caso supervisado, utilizamos una máquina para envolver el modelo no supervisado y los datos:

In [ ]:
v = [1, 2, 3, 4]
stand_model = UnivariateStandardizer()
stand = machine(stand_model, v)

A continuación, podemos ajustar la máquina y utilizarla para aplicar la *transformación de datos* correspondiente:

In [ ]:
fit!(stand)
w = MLJ.transform(stand, v)
@show round.(w, digits=2)
@show mean(w)
@show std(w)

En este caso, el modelo también tiene una transformación inversa:

In [ ]:
vv = inverse_transform(stand, w)
sum(abs.(vv .- v))

# 8. Ajuste del Modelo

## Ajustando un hiperparámetro simple

En MLJ, el ajuste se implementa como una envoltura del modelo.

Tras envolver un modelo en una _estrategia de ajuste_ (por ejemplo, validación cruzada) y vincular el modelo envuelto a los datos en una _máquina_, el ajuste de la máquina inicia una búsqueda de los hiperparámetros óptimos del modelo.

Utilizaremos un clasificador de árbol de decisión y ajustaremos la profundidad máxima del árbol.

Como de costumbre, comenzamos cargando los datos y el modelo:

In [ ]:
using MLJ
using PrettyPrinting
X, y = @load_iris
DecisionTreeClassifier = @load DecisionTreeClassifier pkg=DecisionTree

### Especifica un rango de valores

Para especificar un rango de valores, puedes utilizar la función `range`:

In [ ]:
dtc = DecisionTreeClassifier()
r   = range(dtc, :max_depth, lower=1, upper=5)

Como puedes ver, la función range toma un modelo (`dtc`), un símbolo para el hiperparámetro de interés (`:max_depth`) y la indicación de cómo muestrear los valores.

Para hiperparámetros de tipo `<:Real`, debes especificar un rango de valores como se hizo anteriormente.

Para hiperparámetros de otro tipo (por ejemplo, `Symbol`), debes utilizar la palabra clave `values=...`.

Una vez definido el rango de valores, podemos envolver el modelo en un `TunedModel` especificando la estrategia de ajuste.

In [ ]:
tm = TunedModel(model=dtc, ranges=[r, ], measure=cross_entropy)

Ten en cuenta que "envolver un modelo en una estrategia de ajuste" significa crear una nueva versión "autoajustable" del modelo, `tuned_model = TunedModel(model=...)`, en la que otros argumentos clave especifican:
1. el algoritmo (es decir, la estrategia de ajuste) para buscar el espacio de hiperparámetros del modelo (por ejemplo, `tuning = Random(rng=123)` o `tuning = Grid(goal=100)`).
2. la estrategia de remuestreo, utilizada para evaluar el rendimiento para cada valor de los hiperparámetros (por ejemplo, `resampling=CV(nfolds=9, rng=123)` o `resampling=Holdout(fraction_train=0.7)`).
3. la medida (o medidas) en las que se basarán las evaluaciones de rendimiento (y para la elaboración de informes) (por ejemplo, `measure = rms` o `measures = [rms, mae]`).
4. el rango, que normalmente describe el «espacio» de hiperparámetros a buscar (pero más generalmente cualquier información extra que se requiera para completar la especificación de la búsqueda, por ejemplo, valores iniciales en la optimización de descenso por gradiente).

### Ajustando e inspeccionando un modelo ajustado

Para ajustar un modelo ajustado, puedes utilizar la sintaxis habitual:

In [ ]:
m = machine(tm, X, y)
fit!(m)

Para inspeccionar el mejor modelo, puedes utilizar la función `fitted_params` en la máquina e inspeccionar el campo `best_model`:

In [ ]:
fitted_params(m).best_model.max_depth

Observe que aquí hemos ajustado un modelo probabilístico y, en consecuencia, hemos utilizado una medida probabilística para el ajuste.

También podríamos haber decidido que sólo nos importaba el modo y la tasa de clasificación errónea, para ello, basta con utilizar `operation=predict_mode` en el modelo ajustado:

In [ ]:
tm = TunedModel(model=dtc, ranges=r, operation=predict_mode,
                measure=misclassification_rate)
m = machine(tm, X, y)
fit!(m)
fitted_params(m).best_model.max_depth

Comprobemos la tasa de clasificación errónea del mejor modelo:

In [ ]:
r = report(m)
r.best_history_entry.measurement[1]

In [ ]:
using Plots

plot(m, size=(600,400))

## Ajuste de hyperparámetros anidados

Generemos datos de regresión ficticios simples:

In [227]:
X = (x1=rand(100), x2=rand(100), x3=rand(100))
y = 2X.x1 - X.x2 + 0.05 * randn(100);

Construyamos entonces un modelo ensemble simple con regresores de árbol de decisión:

In [ ]:
DecisionTreeRegressor = @load DecisionTreeRegressor pkg=DecisionTree
forest = EnsembleModel(model=DecisionTreeRegressor())

Un modelo de este tipo tiene hiperparámetros *anidados* en el sentido de que el conjunto tiene hiperparámetros (por ejemplo, `:bagging_fraction`) y cada individuo tiene hiperparámetros (por ejemplo, `:n_subfeatures` o `:max_depth`). Puedes ver esto inspeccionando los parámetros usando `params`:

In [ ]:
params(forest) |> pprint

Los rangos de los hiperparámetros anidados se especifican utilizando la sintaxis de punto, el resto se hace de forma muy parecida a como se hacía antes:

In [ ]:
r1 = range(forest, :(model.n_subfeatures), lower=1, upper=3)
r2 = range(forest, :bagging_fraction, lower=0.4, upper=1.0)
tm = TunedModel(model=forest, tuning=Grid(resolution=12),
                resampling=CV(nfolds=6), ranges=[r1, r2],
                measure=rms)
m = machine(tm, X, y)
fit!(m);

Una función útil para inspeccionar un modelo después de ajustarlo es la función `report` que recoge información sobre el modelo y el ajuste, por ejemplo se puede utilizar para recuperar la mejor medida:

In [ ]:
r = report(m)
r.best_history_entry.measurement[1]

In [ ]:
plot(m)